###### Imports and Settings

In [2]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces

# Decennial SF1 2000

In [4]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [5]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

### SF1

In [6]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE SF12000.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [7]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]

In [8]:
dfs = [dg1, dg2]
droppers = ['ID', 'Category', 'Sub Category']
for df in dfs:
    df.drop(droppers, axis = 1, inplace = True)

In [17]:
url_str= 'https://api.census.gov/data/2000/dec/sf1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
tail_cols3 = 'GeoFIPS2'

In [10]:
results = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list = deque(col_list)
        col_list.appendleft(head2)
        col_list.appendleft(head1)
        predicates= {} #call for all counties in state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:47"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        #df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
        predicates= {} #call for all counties in state of KY
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "county:*"
        predicates["in"]= "state:21"                                                             
        data = requests.get(url_str, params= predicates)                                                                
        col_names = col_list
        kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
        df = pd.concat([df, kycos], axis = 0)
        predicates= {} #call for all places in TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "place:*"
        predicates["in"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places in the region
        df = pd.concat([df, places], axis = 0)
        predicates= {} #call for all places in KY
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "place:*"
        predicates["in"]= "state:21"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        places=places.loc[places['GEO_ID'].isin(censusplaces)] #filter for places of concern in KY
        df = pd.concat([df, places], axis = 0)
        col_list.remove(tail_cols2) #adjust the column list for different formats of geos
        predicates= {} #call for state of TN
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "state:47"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, state], axis = 0)
        predicates= {} #call for US
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "us:*"
        data= requests.get(url_str, params= predicates)
        col_names = col_list
        national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
        df = pd.concat([df, national], axis = 0)
        results.append(df)
new_df = pd.concat(results, axis = 1)
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS', 'GeoFIPS']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)
print('Okay Finished')

Okay Finished


In [11]:
new_df.head(3)

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_total_oneracealone,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,hhtype_total_series,hhtype_oneperson,hhtype_oneperson_male,hhtype_oneperson_female,hhtype_twoormore,hhtype_twoormore_family,hhtype_twoormore_family_marriedcouplefam,hhtype_twoormore_family_marriedcouplefam_ownchildrenunder18,hhtype_twoormore_family_marriedcouplefam_noownchildrenunder18,hhtype_twoormore_family_otherfam,hhtype_twoormore_family_otherfam_malenospouse,hhtype_twoormore_family_otherfam_malenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_malenospouse_noownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse,hhtype_twoormore_family_otherfam_femalenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse_noownchildrenunder18,hhtype_twoormore_nonfamily,hhtype_twoormore_nonfamily_male,hhtype_twoormore_nonfamily_female
0,"Anderson County, Tennessee",0500000US47001,71330,71330,34009,2026,2462,2474,1489,862,375,338,1099,1954,2206,2529,2736,2636,2475,2042,687,929,515,804,1276,1110,614,371,37321,1950,2284,2383,1467,796,378,348,1130,2067,2380,2758,2844,2873,2646,2109,719,1055,657,1010,1743,1628,1101,995,71330,70461,66593,2766,226,593,9,274,869,66124,787,2.37,32451,32451,29780,2671,29780,21592,8188,29780,8259,3198,5061,21521,20513,16024,6321,9703,4489,1063,553,510,3426,1950,1476,1008,608,400
1,"Bedford County, Tennessee",0500000US47003,37586,37586,18645,1370,1384,1361,791,558,308,296,850,1364,1439,1383,1497,1230,1178,931,303,424,265,382,547,400,239,145,18941,1397,1335,1273,776,485,255,251,724,1344,1313,1445,1370,1175,1190,1012,346,472,299,396,668,580,415,420,37586,37149,32640,3189,105,170,20,1025,437,31060,2811,2.67,14990,14990,13905,1085,13905,10226,3679,13905,2987,1230,1757,10918,10350,7968,3405,4563,2382,724,366,358,1658,951,707,568,377,191
2,"Benton County, Tennessee",0500000US47005,16537,16537,8006,434,539,546,328,182,70,87,231,464,508,572,598,568,609,534,199,322,143,257,335,254,122,104,8531,421,543,511,319,179,83,85,243,437,511,597,647,579,619,544,241,255,179,272,417,350,250,249,16537,16423,15948,348,54,40,0,33,114,15834,157,2.37,8595,8595,6863,1732,6863,5527,1336,6863,1766,742,1024,5097,4888,3987,1392,2595,901,250,125,125,651,355,296,209,138,71


In [13]:
part_one = new_df

In [14]:
splits = ['60280', '63140', '48980', '80200', '73460', '29920', '70580']

In [18]:
#split places
data_appended = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list.append(tail_cols3)
        col_list = deque(col_list)
        col_list.appendleft(head2)  
        col_list.appendleft(head1)
        temp = []
        for thingy in splits:
            predicates= {}
            get_vars= var_list
            predicates["get"]= ",". join(get_vars)
            predicates["for"]= 'county (or part):*'
            predicates["in"] = "state:47, place:{}".format(thingy)
            data = requests.get(url_str, params= predicates)
            col_names = col_list
            data = pd.DataFrame(columns=col_names, data=data.json()[1:])
            temp.append(data) #append the current instance to the last
        temp = pd.concat(temp)
        thing = temp
    data_appended.append(temp)
new_df = pd.concat(data_appended, axis = 1) 
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS', 'GeoFIPS', 'GeoFIPS2']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)  
print('Your API call is complete')

Your API call is complete


In [19]:
new_df['GEO_ID'] = '1'
balances = new_df

In [20]:
balances['NAME'].unique()

array(['Sumner County (part), Portland city, Tennessee',
       'Davidson County (part), Ridgetop city, Tennessee',
       'Robertson County (part), Ridgetop city, Tennessee',
       'Robertson County (part), Millersville city, Tennessee',
       'Sumner County (part), Millersville city, Tennessee',
       'Robertson County (part), White House city, Tennessee',
       'Sumner County (part), White House city, Tennessee',
       'Houston County (part), Tennessee Ridge town, Tennessee',
       'Stewart County (part), Tennessee Ridge town, Tennessee',
       'Davidson County (part), Goodlettsville city, Tennessee',
       'Sumner County (part), Goodlettsville city, Tennessee',
       'Maury County (part), Spring Hill city, Tennessee',
       'Williamson County (part), Spring Hill city, Tennessee'],
      dtype=object)

In [29]:
#MSAs are weird.. not sure about how the geography pans out with the consolidated versus primary MSAs... just commenting out for now until it's necessary
#https://api.census.gov/data/2000/dec/sf1/geography.html
msas = ['5360', '1660']

In [31]:
# #this works msa
# data_appended = []
# for df in dfs:
#     dataguide = df
#     for col_name, col_data in df.items():
#         var_list = list(dataguide['Variable']) #make variables list
#         var_list = deque(var_list)
#         var_list.appendleft(head2)
#         var_list.appendleft(head1)
#         col_list = list(dataguide['Column Name']) #make columns list
#         col_list.append(tail_cols1)
#         col_list = deque(col_list)
#         col_list.appendleft(head2)  
#         col_list.appendleft(head1)
#         temp = []
#         for msa in msas:
#             predicates= {}
#             get_vars= var_list
#             predicates["get"]= ",". join(get_vars)
#             predicates["for"]= 'consolidated metropolitan statistical area:{}'.format(msas)
#             data = requests.get(url_str, params= predicates)
#             col_names = col_list
#             data = pd.DataFrame(columns=col_names, data=data.json()[1:])
#             temp.append(data) #append the current instance to the last
#         temp = pd.concat(temp)
#         thing = temp
#     data_appended.append(temp)
# new_df = pd.concat(data_appended, axis = 1) 
# new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
# new_df.columns = new_df.iloc[0]
# new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS']).reset_index(drop = True)
# new_df = new_df.rename_axis(None, axis = 1)    
# print('Your API call is complete')

In [32]:
dfs = [part_one, balances]
data = pd.concat(dfs)
data.head(2)

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_total_oneracealone,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,hhtype_total_series,hhtype_oneperson,hhtype_oneperson_male,hhtype_oneperson_female,hhtype_twoormore,hhtype_twoormore_family,hhtype_twoormore_family_marriedcouplefam,hhtype_twoormore_family_marriedcouplefam_ownchildrenunder18,hhtype_twoormore_family_marriedcouplefam_noownchildrenunder18,hhtype_twoormore_family_otherfam,hhtype_twoormore_family_otherfam_malenospouse,hhtype_twoormore_family_otherfam_malenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_malenospouse_noownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse,hhtype_twoormore_family_otherfam_femalenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse_noownchildrenunder18,hhtype_twoormore_nonfamily,hhtype_twoormore_nonfamily_male,hhtype_twoormore_nonfamily_female
0,"Anderson County, Tennessee",0500000US47001,71330,71330,34009,2026,2462,2474,1489,862,375,338,1099,1954,2206,2529,2736,2636,2475,2042,687,929,515,804,1276,1110,614,371,37321,1950,2284,2383,1467,796,378,348,1130,2067,2380,2758,2844,2873,2646,2109,719,1055,657,1010,1743,1628,1101,995,71330,70461,66593,2766,226,593,9,274,869,66124,787,2.37,32451,32451,29780,2671,29780,21592,8188,29780,8259,3198,5061,21521,20513,16024,6321,9703,4489,1063,553,510,3426,1950,1476,1008,608,400
1,"Bedford County, Tennessee",0500000US47003,37586,37586,18645,1370,1384,1361,791,558,308,296,850,1364,1439,1383,1497,1230,1178,931,303,424,265,382,547,400,239,145,18941,1397,1335,1273,776,485,255,251,724,1344,1313,1445,1370,1175,1190,1012,346,472,299,396,668,580,415,420,37586,37149,32640,3189,105,170,20,1025,437,31060,2811,2.67,14990,14990,13905,1085,13905,10226,3679,13905,2987,1230,1757,10918,10350,7968,3405,4563,2382,724,366,358,1658,951,707,568,377,191


In [33]:
data = data.reset_index(drop = True)
data = data.set_index(['NAME', 'GEO_ID']).transpose()
data.head(2)

NAME,"Anderson County, Tennessee","Bedford County, Tennessee","Benton County, Tennessee","Bledsoe County, Tennessee","Blount County, Tennessee","Bradley County, Tennessee","Campbell County, Tennessee","Cannon County, Tennessee","Carroll County, Tennessee","Carter County, Tennessee","Cheatham County, Tennessee","Chester County, Tennessee","Claiborne County, Tennessee","Clay County, Tennessee","Cocke County, Tennessee","Coffee County, Tennessee","Crockett County, Tennessee","Cumberland County, Tennessee","Davidson County, Tennessee","Decatur County, Tennessee","DeKalb County, Tennessee","Dickson County, Tennessee","Dyer County, Tennessee","Fayette County, Tennessee","Fentress County, Tennessee","Franklin County, Tennessee","Gibson County, Tennessee","Giles County, Tennessee","Grainger County, Tennessee","Greene County, Tennessee","Grundy County, Tennessee","Hamblen County, Tennessee","Hamilton County, Tennessee","Hancock County, Tennessee","Hardeman County, Tennessee","Hardin County, Tennessee","Hawkins County, Tennessee","Haywood County, Tennessee","Henderson County, Tennessee","Henry County, Tennessee","Hickman County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Jackson County, Tennessee","Jefferson County, Tennessee","Johnson County, Tennessee","Knox County, Tennessee","Lake County, Tennessee","Lauderdale County, Tennessee","Lawrence County, Tennessee","Lewis County, Tennessee","Lincoln County, Tennessee","Loudon County, Tennessee","McMinn County, Tennessee","McNairy County, Tennessee","Macon County, Tennessee","Madison County, Tennessee","Marion County, Tennessee","Marshall County, Tennessee","Maury County, Tennessee","Meigs County, Tennessee","Monroe County, Tennessee","Montgomery County, Tennessee","Moore County, Tennessee","Morgan County, Tennessee","Obion County, Tennessee","Overton County, Tennessee","Perry County, Tennessee","Pickett County, Tennessee","Polk County, Tennessee","Putnam County, Tennessee","Rhea County, Tennessee","Roane County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Scott County, Tennessee","Sequatchie County, Tennessee","Sevier County, Tennessee","Shelby County, Tennessee","Smith County, Tennessee","Stewart County, Tennessee","Sullivan County, Tennessee","Sumner County, Tennessee","Tipton County, Tennessee","Trousdale County, Tennessee","Unicoi County, Tennessee","Union County, Tennessee","Van Buren County, Tennessee","Warren County, Tennessee","Washington County, Tennessee","Wayne County, Tennessee","Weakley County, Tennessee","White County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee","Allen County, Kentucky","Christian County, Kentucky","Simpson County, Kentucky","Adams city, Tennessee","Ashland City town, Tennessee","Belle Meade city, Tennessee","Berry Hill city, Tennessee","Brentwood city, Tennessee","Burns town, Tennessee","Cedar Hill city, Tennessee","Charlotte town, Tennessee","Clarksville city, Tennessee","Columbia city, Tennessee","Coopertown town, Tennessee","Cross Plains city, Tennessee","Cumberland City town, Tennessee","Dickson city, Tennessee","Dover city, Tennessee","Eagleville city, Tennessee","Erin city, Tennessee","Fairview city, Tennessee","Forest Hills city, Tennessee","Franklin city, Tennessee","Gallatin city, Tennessee","Goodlettsville city, Tennessee","Greenbrier town, Tennessee","Hendersonville city, Tennessee","Kingston Springs town, Tennessee","Lafayette city, Tennessee","La Vergne city, Tennessee","Lebanon city, Tennessee","McEwen city, Tennessee","Millersville city, Tennessee","Mitchellville city, Tennessee","Mount Juliet city, Tennessee","Mount Pleasant city, Tennessee","Murfreesboro city, Tennessee","Nashville-Davidson (balance), Tennessee","New Johnsonville city, Tennessee","Nolensville town, Tennessee","Oak Hill city, Tennessee","Orlinda town, Tennessee","Pegram town, Tennessee","Pleasant View city, Tennessee","Portland city, Tennessee","Ridgetop city, Tennessee","Slayden town, Tennessee"

In [34]:
numcols = list(data.columns)
numcols
data[numcols] = data[numcols].astype(float)

In [38]:
#no robertson county portland... orlinda was a town
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
thelist = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')], 
                   data[('Christian County, Kentucky', '0500000US21047')]
          ]
data['Fifteen County Region'] = sum(thelist)
#Cheatham
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
#Dickson
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
#Houston
HoustInc = [data[('Erin city, Tennessee', '1600000US4724320')],data[('Houston County (part), Tennessee Ridge town, Tennessee', '1')]]
data[('Houston Incorporated', 'None')] = sum(HoustInc)
data[('Houston Unincorporated', 'None')] = data[('Houston County, Tennessee', '0500000US47083')] - data[('Houston Incorporated', 'None')]
#Humphreys
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
#Maury
MauryInc = [data[('Mount Pleasant city, Tennessee', '1600000US4751080')],data[('Columbia city, Tennessee', '1600000US4716540')],
            data[('Maury County (part), Spring Hill city, Tennessee', '1')]]
data[('Maury Incorporated', 'None')] = sum(MauryInc)
data[('Maury Unincorporated', 'None')] = data[('Maury County, Tennessee', '0500000US47119')] - data[('Maury Incorporated', 'None')]
#Montgomery
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]
#Robertson
RobInc = [data[('Adams city, Tennessee', '1600000US4700200')],data[('Greenbrier town, Tennessee', '1600000US4730960')],
          data[('Cedar Hill city, Tennessee', '1600000US4711980')],data[('Coopertown town, Tennessee', '1600000US4716980')], 
         data[('Cross Plains city, Tennessee', '1600000US4718420')],data[('Springfield city, Tennessee', '1600000US4770500')], 
          data[('Orlinda town, Tennessee', '1600000US4756020')],
         #data[('Robertson County (part), Portland city, Tennessee', '1')],
          data[('Robertson County (part), Millersville city, Tennessee', '1')], 
         data[('Robertson County (part), White House city, Tennessee', '1')], data[('Robertson County (part), Ridgetop city, Tennessee', '1')]]
data[('Robertson Incorporated', 'None')] = sum(RobInc)
data[('Robertson Unincorporated', 'None')] = data[('Robertson County, Tennessee', '0500000US47147')] - data[('Robertson Incorporated', 'None')]
#Rutherford
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
#Stewart
data[('Stewart County (part), Tennessee Ridge town, Tennessee', '1')] = data[('Tennessee Ridge town, Tennessee', '1600000US4773460')] - data[('Houston County (part), Tennessee Ridge town, Tennessee', '1')]
StewInc = [data[('Cumberland City town, Tennessee', '1600000US4718820')],data[('Dover city, Tennessee', '1600000US4721400')],
            data[('Stewart County (part), Tennessee Ridge town, Tennessee', '1')]]
data[('Stewart Incorporated', 'None')] = sum(StewInc)
data[('Stewart Unincorporated', 'None')] = data[('Stewart County, Tennessee', '0500000US47161')] - data[('Stewart Incorporated', 'None')]
#Sumner
SumInc = [data[('Hendersonville city, Tennessee', '1600000US4733280')],data[('Mitchellville city, Tennessee', '1600000US4749460')],
          data[('Westmoreland town, Tennessee', '1600000US4779420')],data[('Gallatin city, Tennessee', '1600000US4728540')], 
         data[('Sumner County (part), Portland city, Tennessee', '1')],data[('Sumner County (part), Millersville city, Tennessee', '1')], 
         data[('Sumner County (part), White House city, Tennessee', '1')],data[('Sumner County (part), Goodlettsville city, Tennessee', '1')]]
data[('Sumner Incorporated', 'None')] = sum(SumInc)
data[('Sumner Unincorporated', 'None')] = data[('Sumner County, Tennessee', '0500000US47165')] - data[('Sumner Incorporated', 'None')]
#Williamson
WilliInc = [data[('Nolensville town, Tennessee', '1600000US4753460')],data[('Brentwood city, Tennessee', '1600000US4708280')],
          data[('Fairview city, Tennessee', '1600000US4725440')],data[('Franklin city, Tennessee', '1600000US4727740')], 
         data[("Thompson's Station town, Tennessee", '1600000US4773900')],data[('Williamson County (part), Spring Hill city, Tennessee', '1')]]
data[('Williamson Incorporated', 'None')] = sum(WilliInc)
data[('Williamson Unincorporated', 'None')] = data[('Williamson County, Tennessee', '0500000US47187')] - data[('Williamson Incorporated', 'None')]
#Wilson
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]

In [39]:
data = data.transpose().reset_index()
data.head()

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_total_oneracealone,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,hhtype_total_series,hhtype_oneperson,hhtype_oneperson_male,hhtype_oneperson_female,hhtype_twoormore,hhtype_twoormore_family,hhtype_twoormore_family_marriedcouplefam,hhtype_twoormore_family_marriedcouplefam_ownchildrenunder18,hhtype_twoormore_family_marriedcouplefam_noownchildrenunder18,hhtype_twoormore_family_otherfam,hhtype_twoormore_family_otherfam_malenospouse,hhtype_twoormore_family_otherfam_malenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_malenospouse_noownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse,hhtype_twoormore_family_otherfam_femalenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse_noownchildrenunder18,hhtype_twoormore_nonfamily,hhtype_twoormore_nonfamily_male,hhtype_twoormore_nonfamily_female
0,"Anderson County, Tennessee",0500000US47001,71330.0,71330.0,34009.0,2026.0,2462.0,2474.0,1489.0,862.0,375.0,338.0,1099.0,1954.0,2206.0,2529.0,2736.0,2636.0,2475.0,2042.0,687.0,929.0,515.0,804.0,1276.0,1110.0,614.0,371.0,37321.0,1950.0,2284.0,2383.0,1467.0,796.0,378.0,348.0,1130.0,2067.0,2380.0,2758.0,2844.0,2873.0,2646.0,2109.0,719.0,1055.0,657.0,1010.0,1743.0,1628.0,1101.0,995.0,71330.0,70461.0,66593.0,2766.0,226.0,593.0,9.0,274.0,869.0,66124.0,787.0,2.37,32451.0,32451.0,29780.0,2671.0,29780.0,21592.0,8188.0,29780.0,8259.0,3198.0,5061.0,21521.0,20513.0,16024.0,6321.0,9703.0,4489.0,1063.0,553.0,510.0,3426.0,1950.0,1476.0,1008.0,608.0,400.0
1,"Bedford County, Tennessee",0500000US47003,37586.0,37586.0,18645.0,1370.0,1384.0,1361.0,791.0,558.0,308.0,296.0,850.0,1364.0,1439.0,1383.0,1497.0,1230.0,1178.0,931.0,303.0,424.0,265.0,382.0,547.0,400.0,239.0,145.0,18941.0,1397.0,1335.0,1273.0,776.0,485.0,255.0,251.0,724.0,1344.0,1313.0,1445.0,1370.0,1175.0,1190.0,1012.0,346.0,472.0,299.0,396.0,668.0,580.0,415.0,420.0,37586.0,37149.0,32640.0,3189.0,105.0,170.0,20.0,1025.0,437.0,31060.0,2811.0,2.67,14990.0,14990.0,13905.0,1085.0,13905.0,10226.0,3679.0,13905.0,2987.0,1230.0,1757.0,10918.0,10350.0,7968.0,3405.0,4563.0,2382.0,724.0,366.0,358.0,1658.0,951.0,707.0,568.0,377.0,191.0
2,"Benton County, Tennessee",0500000US47005,16537.0,16537.0,8006.0,434.0,539.0,546.0,328.0,182.0,70.0,87.0,231.0,464.0,508.0,572.0,598.0,568.0,609.0,534.0,199.0,322.0,143.0,257.0,335.0,254.0,122.0,104.0,8531.0,421.0,543.0,511.0,319.0,179.0,83.0,85.0,243.0,437.0,511.0,597.0,647.0,579.0,619.0,544.0,241.0,255.0,179.0,272.0,417.0,350.0,250.0,249.0,16537.0,16423.0,15948.0,348.0,54.0,40.0,0.0,33.0,114.0,15834.0,157.0,2.37,8595.0,8595.0,6863.0,1732.0,6863.0,5527.0,1336.0,6863.0,1766.0,742.0,1024.0,5097.0,4888.0,3987.0,1392.0,2595.0,901.0,250.0,125.0,125.0,651.0,355.0,296.0,209.0,138.0,71.0
3,"Bledsoe County, Tennessee",0500000US47007,12367.0,12367.0,6771.0,384.0,431.0,425.0,316.0,193.0,69.0,91.0,252.0,499.0,548.0,611.0,621.0,492.0,516.0,392.0,151.0,170.0,100.0,118.0,178.0,112.0,62.0,40.0,5596.0,345.0,363.0,

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 90 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   NAME                                                                  198 non-null    object 
 1   GEO_ID                                                                198 non-null    object 
 2   pop                                                                   198 non-null    float64
 3   agebysex_total_series                                                 198 non-null    float64
 4   age_total_male                                                        198 non-null    float64
 5   age_m_u5                                                              198 non-null    float64
 6   age_m_5to9                                                            198 non-null    float64
 7  

In [41]:
data.to_feather('../../Raw Data/SF12000')